In [3]:
from dask.distributed import Client
import pandas as pd
import pyarrow.parquet as pq

## Словарь юрл, доля мужчин. Пропуски заполним 0.5

In [ ]:
df = pq.read_table('data/train_data_pqt.parquet',columns=['user_id','request_cnt','url_host'])
target = pd.read_parquet('data/public_train.pqt')

data_id_url = df.select(['user_id', 'url_host', 'request_cnt']).to_pandas()

data_id_url_sum_male = data_id_url.merge(target[['is_male','user_id']], on = 'user_id', how = 'inner')
data_id_url_sum_male = data_id_url_sum_male.loc[~(data_id_url_sum_male['is_male'].isna()) & (data_id_url_sum_male['is_male'] != 'NA')]
data_id_url_sum_male['is_male'] = data_id_url_sum_male['is_male'].astype('int8')

data_id_url_sum_male['male_request_cnt'] = data_id_url_sum_male['is_male'] * data_id_url_sum_male['request_cnt']
data_id_url_sum_male = data_id_url_sum_male.groupby('url_host').agg({'request_cnt':'sum','male_request_cnt':'sum'})
data_id_url_sum_male['male_fraction'] = data_id_url_sum_male['male_request_cnt']/data_id_url_sum_male['request_cnt']
display(data_id_url_sum_male.sort_values('request_cnt').tail())
display(data_id_url_sum_male.info())

## Словарь регион, доля мужчин. Пропуски заполним 0.5

In [7]:
df = pq.read_table('data/train_data_pqt.parquet',columns=['user_id','region_name'])
target = pd.read_parquet('data/public_train.pqt')

data_id_reg = df.select(['user_id','region_name']).to_pandas()

data_id_reg_male = data_id_reg.merge(target[['is_male','user_id']], on = 'user_id', how = 'inner')
data_id_reg_male = data_id_reg_male.loc[~(data_id_reg_male['is_male'].isna()) & (data_id_reg_male['is_male'] != 'NA')]
data_id_reg_male['is_male'] = data_id_reg_male['is_male'].astype('int8')

data_id_reg_male = data_id_reg_male.groupby('region_name').agg({'is_male': 'mean'})
data_id_reg_male

,is_male
region_name,
Алтайский край,0.507297
Архангельская область,0.484938
Владимирская область,0.519503
Вологодская область,0.528699
Воронежская область,0.581428
...,...
Республика Тыва,0.395121
Камчатский край,0.536116
Республика Калмыкия,0.381443


## Суммируем заходы в инет утром днем ночью и вечером

In [14]:
df = pq.read_table('data/train_data_pqt.parquet',columns=['user_id','request_cnt', 'part_of_day'])
target = pd.read_parquet('data/public_train.pqt')

data_day_req = df.select(['user_id','request_cnt', 'part_of_day']).to_pandas()

data_day_req.groupby(['user_id','part_of_day'],group_keys=False)['request_cnt'].sum()

user_id  part_of_day
0        day            107
         evening         62
         morning         23
         night            1
1        day            363
                       ... 
415312   night            0
415313   day              1
         evening          0
         morning          0
         night            0
Name: request_cnt, Length: 1079996, dtype: int64